In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8419.76it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4670.32it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2626848.03it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9224.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4970.49it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 76389.45it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9179.06it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x111beb4e0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x135b2b320>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (100,)]}

In [5]:
model = mz.models.DUET()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['lm_filters'] = 32
model.params['lm_hidden_sizes'] = [32]
model.params['dm_filters'] = 32
model.params['dm_kernel_size'] = 3
model.params['dm_d_mpool'] = 4
model.params['dm_hidden_sizes'] = [32]
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = 'adagrad'

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.duet.DUET'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adagrad
with_embedding                True
embedding_input_dim           16674
embedding_output_dim          100
embedding_trainable           True
lm_filters                    32
lm_hidden_sizes               [32]
dm_filters                    32
dm_kernel_size                3
dm_q_hidden_size              32
dm_d_mpool                    4
dm_hidden_sizes               [32]
padding                       same
activation_func               relu
dropout_rate                  0.5


In [6]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 10, 32)       9632        embedding[0][0]                  
__________

In [7]:
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
test_x, test_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_x))

In [10]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=4, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 4s 39ms/step - loss: 0.8772
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4580984845812892 - normalized_discounted_cumulative_gain@5(0.0): 0.5292475531391599 - mean_average_precision(0.0): 0.49045044430509527
Epoch 2/30
102/102 [==============================] - 6s 56ms/step - loss: 0.6112
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5702800514952101 - normalized_discounted_cumulative_gain@5(0.0): 0.6329332893788385 - mean_average_precision(0.0): 0.585411336972796
Epoch 3/30
102/102 [==============================] - 6s 59ms/step - loss: 0.3512
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5520530433717853 - normalized_discounted_cumulative_gain@5(0.0): 0.6166989840324851 - mean_average_precision(0.0): 0.5661811400902007
Epoch 4/30
102/102 [==============================] - 6s 64ms/step - loss: 0.2440
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5467697804683268 - normalized_

In [12]:
append_params_to_readme(model)